In [48]:
from policyengine_core.model_api import *
from policyengine_us import *
from scipy.optimize import differential_evolution

INCOME_ELASTICITY = -0.05
SUBSTITUTION_ELASTICITY = 0.25

def create_funding_reform(flat_tax_rate: float):
    return Reform.from_dict({
        "gov.contrib.ubi_center.flat_tax.abolish_federal_income_tax": {
            "year:2023:10": True
        },
        "gov.contrib.ubi_center.flat_tax.abolish_payroll_tax": {
            "year:2023:10": True
        },
        "gov.contrib.ubi_center.flat_tax.abolish_self_emp_tax": {
            "year:2023:10": True
        },
        "gov.ssa.ssi.abolish_ssi": {
            "year:2023:10": True
        },
        "gov.usda.snap.abolish_snap": {
            "year:2023:10": True
        },
        "gov.usda.wic.abolish_wic": {
            "year:2023:10": True
        },
        "gov.contrib.ubi_center.flat_tax.rate": {
            "year:2023:10": flat_tax_rate
        },
        "gov.contrib.ubi_center.flat_tax.deduct_ptc": {
            "year:2023:10": True
        },
        "gov.usda.snap.emergency_allotment.allowed": {
            "year:2023:10": False
        },
        "simulation.reported_state_income_tax": {
            "year:2023:10": True
        },
        "gov.simulation.labor_supply_responses.income_elasticity": {
            "year:2023:10": INCOME_ELASTICITY
        },
        "gov.simulation.labor_supply_responses.substitution_elasticity": {
            "year:2023:10": SUBSTITUTION_ELASTICITY
        },
    },
    "us",
    )

def create_baseline_reform():
    return Reform.from_dict({
            "gov.usda.snap.emergency_allotment.allowed": {
                "year:2023:10": False
            },
            "simulation.reported_state_income_tax": {
                "year:2023:10": True
            },
        },
        "us",
    )


class BlankSlatePolicy:
    young_child: float = 0
    older_child: float = 0
    young_adult: float = 0
    adult: float = 0
    senior: float = 0
    flat_tax_rate: float = 0.40

    def __init__(self, flat_tax_rate: float = 0.40):
        self.baseline = Microsimulation(reform=create_baseline_reform(), dataset="enhanced_cps_2023")
        self.blank_slate_funded = Microsimulation(reform=create_funding_reform(flat_tax_rate), dataset="enhanced_cps_2023")
        self.df = self.create_dataframe()
        self.ubi_funding = self.get_ubi_funding()
        self.flat_tax_rate = flat_tax_rate

    def create_dataframe(self) -> pd.DataFrame:
        age = self.baseline.calc("age").values
        df = pd.DataFrame(
            dict(
                baseline_net_income=self.baseline.calc(
                    "household_net_income"
                ).values,
                count_young_child=self.baseline.map_result(
                    age < 6, "person", "household"
                ),
                count_older_child=self.baseline.map_result(
                    (age >= 6) & (age < 18), "person", "household"
                ),
                count_young_adult=self.baseline.map_result(
                    (age >= 18) & (age < 25), "person", "household"
                ),
                count_adult=self.baseline.map_result(
                    (age >= 25) & (age < 65), "person", "household"
                ),
                count_senior=self.baseline.map_result(
                    age >= 65, "person", "household"
                ),
                count_person=self.baseline.map_result(
                    age >= 0, "person", "household"
                ),
                funded_net_income=self.blank_slate_funded.calculate(
                    "household_net_income"
                ).values,
                weight=self.baseline.calculate("household_weight").values,
            )
        )

        df["total_employment_income"] = self.baseline.calculate("employment_income", map_to="household").values
        return df

    def get_ubi_funding(self) -> float:
        return (
            (self.df.baseline_net_income - self.df.funded_net_income)
            * self.df.weight
        ).sum()

    def get_senior_amount(
        self,
        young_child: float,
        older_child: float,
        young_adult: float,
        adult: float,
        funding_offset: float = 0,
    ) -> float:
        return (
            self.ubi_funding - funding_offset
            - young_child * (self.df.count_young_child * self.df.weight).sum()
            - older_child * (self.df.count_older_child * self.df.weight).sum()
            - young_adult * (self.df.count_young_adult * self.df.weight).sum()
            - adult * (self.df.count_adult * self.df.weight).sum()
        ) / (self.df.count_senior * self.df.weight).sum()

    def mean_percentage_loss(
        self,
        young_child: float,
        older_child: float,
        young_adult: float,
        adult: float,
        return_extras: bool = False
    ) -> float:
        senior_amount = self.get_senior_amount(
            young_child, older_child, young_adult, adult
        )
        final_net_income = (
            self.df.funded_net_income
            + self.df.count_young_child * young_child
            + self.df.count_older_child * older_child
            + self.df.count_young_adult * young_adult
            + self.df.count_adult * adult
            + self.df.count_senior * senior_amount
        )
        gain = final_net_income - self.df.baseline_net_income
        income_rel_change = final_net_income / self.df.funded_net_income - 1
        income_rel_change_c = np.clip(income_rel_change, -1, 1)
        income_effect = income_rel_change_c * self.df.total_employment_income * INCOME_ELASTICITY
        employment_income_change = income_effect
        MTR_GUESS = 0.3
        rough_net_income_change_from_lsr = employment_income_change * (1 - MTR_GUESS)
        rough_tax_change_from_lsr = employment_income_change * MTR_GUESS
        total_ubi_funding_change = (rough_tax_change_from_lsr * self.df.weight).sum()
        senior_amount = self.get_senior_amount(
            young_child, older_child, young_adult, adult, total_ubi_funding_change
        )
        final_net_income = (
            self.df.funded_net_income
            + self.df.count_young_child * young_child
            + self.df.count_older_child * older_child
            + self.df.count_young_adult * young_adult
            + self.df.count_adult * adult
            + self.df.count_senior * senior_amount
        )
        gain = final_net_income - self.df.baseline_net_income + rough_net_income_change_from_lsr
        absolute_loss = np.maximum(0, -gain)
        pct_loss = absolute_loss / np.maximum(100, self.df.baseline_net_income)
        average = np.average(
            pct_loss, weights=self.df.weight * self.df.count_person
        )
        if return_extras:
            return average, senior_amount, total_ubi_funding_change
        return average

    def solve(self, return_amounts: bool = False, return_loss: bool = False) -> dict:
        (
            self.young_child,
            self.older_child,
            self.young_adult,
            self.adult,
        ) = differential_evolution(
            lambda x: self.mean_percentage_loss(*x),
            bounds=[(0, 30e3 * self.flat_tax_rate)] * 4,
            maxiter=int(1e2),
            seed=0,
        ).x
        _, self.senior, _ = self.mean_percentage_loss(
            self.young_child, self.older_child, self.young_adult, self.adult, return_extras=True
        )
        self.reform = Reform.from_dict(
            {
                **create_funding_reform(self.flat_tax_rate).parameter_values,
                "gov.contrib.ubi_center.basic_income.amount.person.by_age[0].amount": {
                    "year:2023:10": self.young_child
                },
                "gov.contrib.ubi_center.basic_income.amount.person.by_age[1].amount": {
                    "year:2023:10": self.older_child
                },
                "gov.contrib.ubi_center.basic_income.amount.person.by_age[2].amount": {
                    "year:2023:10": self.young_adult
                },
                "gov.contrib.ubi_center.basic_income.amount.person.by_age[3].amount": {
                    "year:2023:10": self.adult
                },
                "gov.contrib.ubi_center.basic_income.amount.person.by_age[4].amount": {
                    "year:2023:10": self.senior
                },
            },
            "us",
        )
        if not return_amounts and not return_loss:
            return self.reform
        
        data = dict(reform=self.reform)

        if return_amounts:
            data["amounts"] = dict(
                young_child=self.young_child,
                older_child=self.older_child,
                young_adult=self.young_adult,
                adult=self.adult,
                senior=self.senior,
            )
        
        if return_loss:
            data["loss"] = self.mean_percentage_loss(
                self.young_child, self.older_child, self.young_adult, self.adult
            )
        
        public_reform = Reform.from_dict(
            {
                path: value for path, value in self.reform.parameter_values.items()
                if path.startswith("gov")
            },
            "us",
        )
        #data["id"] = public_reform.api_id
        data["reform"] = public_reform
        
        return data
"""
from tqdm import tqdm

flat_tax_rates = []
api_ids = []
reforms = []
losses = []

for flat_tax in tqdm(np.linspace(0.3, 0.3, 1)):
    policy = BlankSlatePolicy(flat_tax)
    data = policy.solve(return_amounts=True, return_loss=True)
    flat_tax_rates.append(flat_tax)
    #api_ids.append(data["id"])
    reforms.append(data["reform"])
    losses.append(data["loss"])

df = pd.DataFrame(dict(flat_tax_rate=flat_tax_rates, loss=losses))
"""

'\nfrom tqdm import tqdm\n\nflat_tax_rates = []\napi_ids = []\nreforms = []\nlosses = []\n\nfor flat_tax in tqdm(np.linspace(0.3, 0.3, 1)):\n    policy = BlankSlatePolicy(flat_tax)\n    data = policy.solve(return_amounts=True, return_loss=True)\n    flat_tax_rates.append(flat_tax)\n    #api_ids.append(data["id"])\n    reforms.append(data["reform"])\n    losses.append(data["loss"])\n\ndf = pd.DataFrame(dict(flat_tax_rate=flat_tax_rates, loss=losses))\n'

In [49]:
policy = BlankSlatePolicy(0.3)

In [ ]:
policy.df

,baseline_net_income,count_young_child,count_older_child,count_young_adult,count_adult,count_senior,count_person,funded_net_income,weight,total_employment_income
0,2.849216e+04,0.0,0.0,0.0,0.0,2.0,2.0,2.822000e+04,1554.098145,0.000000e+00
1,4.637827e+04,0.0,0.0,0.0,2.0,0.0,2.0,3.562937e+04,2515.147705,4.200000e+04
2,1.664416e+04,0.0,0.0,0.0,0.0,1.0,1.0,1.637180e+04,1263.665649,0.000000e+00
3,1.014171e+05,0.0,0.0,0.0,0.0,2.0,2.0,8.622158e+04,1590.238525,1.070000e+05
4,4.370309e+04,0.0,0.0,0.0,0.0,2.0,2.0,4.040963e+04,1727.838135,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...
113673,2.565957e+05,0.0,1.0,1.0,2.0,0.0,4.0,2.338610e+05,0.000000,3.306100e+05
113674,2.576290e+05,0.0,0.0,0.0,2.0,0.0,2.0,2.396548e+05,0.000000,2.165010e+05
113675,3.373606e+05,0.0,2.0,0.0,2.0,2.0,6.0,2.960637e+05,0.000000,3.476580e+05
113676,1.088401e+07,0.0,2.0,1.0,2.0,0.0,5.0,1.363628e+07,0.000000,2.041000e+07


In [47]:
data = policy.solve(return_amounts=True, return_loss=True)

AttributeError: 'DataFrame' object has no attribute 'reformed_mtr'

In [33]:
data

{'reform': policyengine_core.reforms.reform.Reform.from_dict.<locals>.reform,
 'amounts': {'young_child': 3237.7089459591953,
  'older_child': 3017.637297485982,
  'young_adult': 3999.307512656774,
  'adult': 4281.81795818974,
  'senior': 3444.625036429992},
 'loss': 0.054036227650283854}

In [6]:
import plotly.express as px
from policyengine_core.charts import *

fig = px.line(
    df.sort_values("flat_tax_rate"),
    x="flat_tax_rate",
    y="loss",
)


format_fig(fig).update_traces(
    # Set color to BLUE
    line=dict(color=BLUE_PRIMARY),
).update_layout(
    title="Mean percentage loss by flat tax rate",
    xaxis_title="Flat tax rate",
    yaxis_title="Loss",
    xaxis_tickformat=".0%",
    yaxis_tickformat=".1%",
)

In [27]:
df

,flat_tax_rate,loss
0,0.3,0.025409


In [34]:
data

{'reform': policyengine_core.reforms.reform.Reform.from_dict.<locals>.reform,
 'amounts': {'young_child': 3237.7089459591953,
  'older_child': 3017.637297485982,
  'young_adult': 3999.307512656774,
  'adult': 4281.81795818974,
  'senior': 3444.625036429992},
 'loss': 0.054036227650283854}

In [36]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
from policyengine_core.periods import instant
import pandas as pd


def modify_parameters(parameters):
    parameters.gov.contrib.ubi_center.basic_income.amount.person.by_age[0].amount.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=data["amounts"]["young_child"])
    parameters.gov.contrib.ubi_center.basic_income.amount.person.by_age[1].amount.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=data["amounts"]["older_child"])
    parameters.gov.contrib.ubi_center.basic_income.amount.person.by_age[2].amount.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=data["amounts"]["young_adult"])
    parameters.gov.contrib.ubi_center.basic_income.amount.person.by_age[3].amount.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=data["amounts"]["adult"])
    parameters.gov.contrib.ubi_center.basic_income.amount.person.by_age[4].amount.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=data["amounts"]["senior"])
    parameters.gov.contrib.ubi_center.flat_tax.abolish_federal_income_tax.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=True)
    parameters.gov.contrib.ubi_center.flat_tax.abolish_payroll_tax.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=True)
    parameters.gov.contrib.ubi_center.flat_tax.abolish_self_emp_tax.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=True)
    parameters.gov.contrib.ubi_center.flat_tax.deduct_ptc.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=True)
    parameters.gov.contrib.ubi_center.flat_tax.rate.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=0.1)
    parameters.gov.simulation.labor_supply_responses.income_elasticity.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=-0.05)
    parameters.gov.simulation.labor_supply_responses.substitution_elasticity.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=0.25)
    parameters.gov.ssa.ssi.abolish_ssi.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=True)
    parameters.gov.usda.snap.abolish_snap.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=True)
    parameters.gov.usda.snap.emergency_allotment.allowed.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=False)
    parameters.gov.usda.wic.abolish_wic.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=True)
    parameters.simulation.reported_state_income_tax.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=True)
    return parameters

class reform(Reform):
    def apply(self):
        self.modify_parameters(modify_parameters)


def baseline_modify_parameters(parameters):
    parameters.simulation.reported_state_income_tax.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=True)
    return parameters

class baseline_reform(Reform):
    def apply(self):
        self.modify_parameters(baseline_modify_parameters)


baseline = Microsimulation(reform=baseline_reform)
reformed = Microsimulation(reform=reform)

In [41]:
reformed.calculate("employment_income").sum()/1e9 - baseline.calculate("employment_income").sum()/1e9

750.1940368158139